In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import folium
from folium.plugins import MiniMap
from geopy.geocoders import Nominatim
import time
import requests

In [15]:
house = pd.read_csv('./data/아파트실거래가.csv', encoding='EUC-KR')
house

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,12,"220,000",4,1987,언주로 3,NaN,,중개거래,서울 강남구
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202204,21,"220,000",2,1987,언주로 3,NaN,,중개거래,서울 강남구
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202205,27,"216,000",2,1987,언주로 3,NaN,,중개거래,서울 강남구
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,102.32,202204,1,"369,000",13,2020,개포로 264,NaN,,중개거래,"서울 강남구, 서울 양천구"
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,136.06,202205,2,"420,000",17,2020,개포로 264,NaN,,중개거래,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12681,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202203,27,"73,000",20,1997,동일로 752,NaN,,중개거래,서울 중랑구
12682,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),59.76,202207,20,"74,000",3,1997,동일로 752,NaN,,중개거래,서울 중랑구
12683,서울특별시 중랑구 중화동,450,450.0,0.0,한신아파트(103~109),84.03,202207,27,"91,500",12,1997,동일로 752,NaN,,중개거래,서울 중랑구
12684,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),84.69,202204,9,"49,900",7,2003,동일로144길 74,NaN,,중개거래,서울 중랑구


In [17]:
house.drop(['번지', '본번', '부번', '층', '계약년월', '계약일' , '층',  '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지'] ,axis=1, inplace=True)
house.columns

Index(['시군구', '단지명', '전용면적(㎡)', '거래금액(만원)', '건축년도', '도로명'], dtype='object')

In [18]:
house

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",1987,언주로 3
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"220,000",1987,언주로 3
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,"216,000",1987,언주로 3
3,서울특별시 강남구 개포동,개포래미안포레스트,102.32,"369,000",2020,개포로 264
4,서울특별시 강남구 개포동,개포래미안포레스트,136.06,"420,000",2020,개포로 264
...,...,...,...,...,...,...
12681,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"73,000",1997,동일로 752
12682,서울특별시 중랑구 중화동,한신아파트(103~109),59.76,"74,000",1997,동일로 752
12683,서울특별시 중랑구 중화동,한신아파트(103~109),84.03,"91,500",1997,동일로 752
12684,서울특별시 중랑구 중화동,한영(101),84.69,"49,900",2003,동일로144길 74


시군구에서 구 분리

In [20]:
for index, row in house.iterrows():
    address = row['시군구']
    parts = address.split(' ')  # 공백으로 문자열 분할
    if len(parts) > 2:
        house.at[index, '시군구'] = parts[1]

house

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명
0,강남구,개포6차우성아파트1동~8동,79.97,"220,000",1987,언주로 3
1,강남구,개포6차우성아파트1동~8동,79.97,"220,000",1987,언주로 3
2,강남구,개포6차우성아파트1동~8동,79.97,"216,000",1987,언주로 3
3,강남구,개포래미안포레스트,102.32,"369,000",2020,개포로 264
4,강남구,개포래미안포레스트,136.06,"420,000",2020,개포로 264
...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.76,"73,000",1997,동일로 752
12682,중랑구,한신아파트(103~109),59.76,"74,000",1997,동일로 752
12683,중랑구,한신아파트(103~109),84.03,"91,500",1997,동일로 752
12684,중랑구,한영(101),84.69,"49,900",2003,동일로144길 74


In [23]:
house_dangi_dongbuk = house[(house['시군구'] == '강북구') | (house['시군구'] == '도봉구') | (house['시군구'] == '노원구')| (house['시군구'] == '중랑구')
                            | (house['시군구'] == '성북구')| (house['시군구'] == '동대문구')]
house_dangi_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명
1372,강북구,경남아너스빌(1356-0),114.922,"89,000",2003,오패산로30길 30
1373,강북구,경남아너스빌(1356-0),59.964,"77,000",2003,오패산로30길 30
1374,강북구,경남아너스빌(1356-0),84.873,"85,000",2003,오패산로30길 30
1375,강북구,꿈의숲롯데캐슬,84.980,"113,000",2017,오현로 56
1376,강북구,꿈의숲롯데캐슬,104.530,"117,000",2017,오현로 56
...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,"73,000",1997,동일로 752
12682,중랑구,한신아파트(103~109),59.760,"74,000",1997,동일로 752
12683,중랑구,한신아파트(103~109),84.030,"91,500",1997,동일로 752
12684,중랑구,한영(101),84.690,"49,900",2003,동일로144길 74


In [24]:
house_dangi_dongbuk['주소'] = house_dangi_dongbuk['시군구'] + ' ' + house_dangi_dongbuk['도로명']
house_dangi_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소
1372,강북구,경남아너스빌(1356-0),114.922,"89,000",2003,오패산로30길 30,강북구 오패산로30길 30
1373,강북구,경남아너스빌(1356-0),59.964,"77,000",2003,오패산로30길 30,강북구 오패산로30길 30
1374,강북구,경남아너스빌(1356-0),84.873,"85,000",2003,오패산로30길 30,강북구 오패산로30길 30
1375,강북구,꿈의숲롯데캐슬,84.980,"113,000",2017,오현로 56,강북구 오현로 56
1376,강북구,꿈의숲롯데캐슬,104.530,"117,000",2017,오현로 56,강북구 오현로 56
...,...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,"73,000",1997,동일로 752,중랑구 동일로 752
12682,중랑구,한신아파트(103~109),59.760,"74,000",1997,동일로 752,중랑구 동일로 752
12683,중랑구,한신아파트(103~109),84.030,"91,500",1997,동일로 752,중랑구 동일로 752
12684,중랑구,한영(101),84.690,"49,900",2003,동일로144길 74,중랑구 동일로144길 74


In [25]:
# 카카오맵 API Key
kakao_api_key = '146c824df784569973cb17777ddd3301'

# '위도'와 '경도' 칼럼을 추가합니다.
house_dangi_dongbuk['위도'] = None
house_dangi_dongbuk['경도'] = None

# 카카오맵 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in house_dangi_dongbuk.iterrows():
    address = row['주소']
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('documents'):
            latitude = data['documents'][0]['y']
            longitude = data['documents'][0]['x']
            house_dangi_dongbuk.at[index, '위도'] = latitude
            house_dangi_dongbuk.at[index, '경도'] = longitude

# 결과를 출력합니다.
house_dangi_dongbuk


,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도
1372,강북구,경남아너스빌(1356-0),114.922,"89,000",2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839
1373,강북구,경남아너스빌(1356-0),59.964,"77,000",2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839
1374,강북구,경남아너스빌(1356-0),84.873,"85,000",2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839
1375,강북구,꿈의숲롯데캐슬,84.980,"113,000",2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918
1376,강북구,꿈의숲롯데캐슬,104.530,"117,000",2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918
...,...,...,...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,"73,000",1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444
12682,중랑구,한신아파트(103~109),59.760,"74,000",1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444
12683,중랑구,한신아파트(103~109),84.030,"91,500",1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444
12684,중랑구,한영(101),84.690,"49,900",2003,동일로144길 74,중랑구 동일로144길 74,37.6065141890726,127.082176614989


In [27]:
# 카카오맵 API Key
kakao_api_key = '146c824df784569973cb17777ddd3301'

house_dangi_dongbuk['행정동'] = None

# 카카오맵 API를 호출하여 주소별 위도, 경도 및 행정동명을 얻습니다.
for index, row in house_dangi_dongbuk.iterrows():
    latitude = row['위도']
    longitude = row['경도']
    
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('documents'):
            administrative_district = data['documents'][1]['region_3depth_name']
            
            house_dangi_dongbuk.at[index, '행정동'] = administrative_district

# 결과를 출력합니다. 
house_dangi_dongbuk


,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동
1372,강북구,경남아너스빌(1356-0),114.922,"89,000",2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동
1373,강북구,경남아너스빌(1356-0),59.964,"77,000",2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동
1374,강북구,경남아너스빌(1356-0),84.873,"85,000",2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동
1375,강북구,꿈의숲롯데캐슬,84.980,"113,000",2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동
1376,강북구,꿈의숲롯데캐슬,104.530,"117,000",2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동
...,...,...,...,...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,"73,000",1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동
12682,중랑구,한신아파트(103~109),59.760,"74,000",1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동
12683,중랑구,한신아파트(103~109),84.030,"91,500",1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동
12684,중랑구,한영(101),84.690,"49,900",2003,동일로144길 74,중랑구 동일로144길 74,37.6065141890726,127.082176614989,중화1동


In [28]:
house_dangi_dongbuk['거래금액(만원)'] = house_dangi_dongbuk['거래금액(만원)'].str.replace(',', '', regex=True)
house_dangi_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동
1372,강북구,경남아너스빌(1356-0),114.922,89000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동
1373,강북구,경남아너스빌(1356-0),59.964,77000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동
1374,강북구,경남아너스빌(1356-0),84.873,85000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동
1375,강북구,꿈의숲롯데캐슬,84.980,113000,2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동
1376,강북구,꿈의숲롯데캐슬,104.530,117000,2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동
...,...,...,...,...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,73000,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동
12682,중랑구,한신아파트(103~109),59.760,74000,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동
12683,중랑구,한신아파트(103~109),84.030,91500,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동
12684,중랑구,한영(101),84.690,49900,2003,동일로144길 74,중랑구 동일로144길 74,37.6065141890726,127.082176614989,중화1동


In [29]:
house_dangi_dongbuk['면적(㎡)당금액'] = (house_dangi_dongbuk['거래금액(만원)'].astype(int) / house_dangi_dongbuk['전용면적(㎡)'].astype(int)).round()
house_dangi_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동,면적(㎡)당금액
1372,강북구,경남아너스빌(1356-0),114.922,89000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동,781.0
1373,강북구,경남아너스빌(1356-0),59.964,77000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동,1305.0
1374,강북구,경남아너스빌(1356-0),84.873,85000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동,1012.0
1375,강북구,꿈의숲롯데캐슬,84.980,113000,2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동,1345.0
1376,강북구,꿈의숲롯데캐슬,104.530,117000,2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동,1125.0
...,...,...,...,...,...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,73000,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동,1237.0
12682,중랑구,한신아파트(103~109),59.760,74000,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동,1254.0
12683,중랑구,한신아파트(103~109),84.030,91500,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동,1089.0
12684,중랑구,한영(101),84.690,49900,2003,동일로144길 74,중랑구 동일로144길 74,37.6065141890726,127.082176614989,중화1동,594.0


In [30]:
house_dangi_dongbuk.to_csv('./data/h동북부실거래가전처리.csv')
house_dangi_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동,면적(㎡)당금액
1372,강북구,경남아너스빌(1356-0),114.922,89000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동,781.0
1373,강북구,경남아너스빌(1356-0),59.964,77000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동,1305.0
1374,강북구,경남아너스빌(1356-0),84.873,85000,2003,오패산로30길 30,강북구 오패산로30길 30,37.6163555759387,127.035671190839,송중동,1012.0
1375,강북구,꿈의숲롯데캐슬,84.980,113000,2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동,1345.0
1376,강북구,꿈의숲롯데캐슬,104.530,117000,2017,오현로 56,강북구 오현로 56,37.6184115650264,127.036475927918,송중동,1125.0
...,...,...,...,...,...,...,...,...,...,...,...
12681,중랑구,한신아파트(103~109),59.760,73000,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동,1237.0
12682,중랑구,한신아파트(103~109),59.760,74000,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동,1254.0
12683,중랑구,한신아파트(103~109),84.030,91500,1997,동일로 752,중랑구 동일로 752,37.5970090851956,127.082070702444,중화1동,1089.0
12684,중랑구,한영(101),84.690,49900,2003,동일로144길 74,중랑구 동일로144길 74,37.6065141890726,127.082176614989,중화1동,594.0


In [63]:
house_dongbuk = pd.read_csv('./data/h동북부실거래가전처리.csv')
house_dongbuk = house_dongbuk.drop(columns=['Unnamed: 0'], axis=1)
house_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동,면적(㎡)당금액
0,강북구,경남아너스빌(1356-0),114.922,89000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,781.0
1,강북구,경남아너스빌(1356-0),59.964,77000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,1305.0
2,강북구,경남아너스빌(1356-0),84.873,85000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,1012.0
3,강북구,꿈의숲롯데캐슬,84.980,113000,2017,오현로 56,강북구 오현로 56,37.618412,127.036476,송중동,1345.0
4,강북구,꿈의숲롯데캐슬,104.530,117000,2017,오현로 56,강북구 오현로 56,37.618412,127.036476,송중동,1125.0
...,...,...,...,...,...,...,...,...,...,...,...
3204,중랑구,한신아파트(103~109),59.760,73000,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1237.0
3205,중랑구,한신아파트(103~109),59.760,74000,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1254.0
3206,중랑구,한신아파트(103~109),84.030,91500,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1089.0
3207,중랑구,한영(101),84.690,49900,2003,동일로144길 74,중랑구 동일로144길 74,37.606514,127.082177,중화1동,594.0


In [64]:
house_dongbuk.columns

Index(['시군구', '단지명', '전용면적(㎡)', '거래금액(만원)', '건축년도', '도로명', '주소', '위도', '경도',
       '행정동', '면적(㎡)당금액'],
      dtype='object')

In [65]:
house_dongbuk

,시군구,단지명,전용면적(㎡),거래금액(만원),건축년도,도로명,주소,위도,경도,행정동,면적(㎡)당금액
0,강북구,경남아너스빌(1356-0),114.922,89000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,781.0
1,강북구,경남아너스빌(1356-0),59.964,77000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,1305.0
2,강북구,경남아너스빌(1356-0),84.873,85000,2003,오패산로30길 30,강북구 오패산로30길 30,37.616356,127.035671,송중동,1012.0
3,강북구,꿈의숲롯데캐슬,84.980,113000,2017,오현로 56,강북구 오현로 56,37.618412,127.036476,송중동,1345.0
4,강북구,꿈의숲롯데캐슬,104.530,117000,2017,오현로 56,강북구 오현로 56,37.618412,127.036476,송중동,1125.0
...,...,...,...,...,...,...,...,...,...,...,...
3204,중랑구,한신아파트(103~109),59.760,73000,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1237.0
3205,중랑구,한신아파트(103~109),59.760,74000,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1254.0
3206,중랑구,한신아파트(103~109),84.030,91500,1997,동일로 752,중랑구 동일로 752,37.597009,127.082071,중화1동,1089.0
3207,중랑구,한영(101),84.690,49900,2003,동일로144길 74,중랑구 동일로144길 74,37.606514,127.082177,중화1동,594.0


In [66]:
house_dongbuk.drop(['전용면적(㎡)', '거래금액(만원)',  '건축년도', '도로명', '주소', '위도', '경도'] ,axis=1, inplace=True)
house_dongbuk

,시군구,단지명,행정동,면적(㎡)당금액
0,강북구,경남아너스빌(1356-0),송중동,781.0
1,강북구,경남아너스빌(1356-0),송중동,1305.0
2,강북구,경남아너스빌(1356-0),송중동,1012.0
3,강북구,꿈의숲롯데캐슬,송중동,1345.0
4,강북구,꿈의숲롯데캐슬,송중동,1125.0
...,...,...,...,...
3204,중랑구,한신아파트(103~109),중화1동,1237.0
3205,중랑구,한신아파트(103~109),중화1동,1254.0
3206,중랑구,한신아파트(103~109),중화1동,1089.0
3207,중랑구,한영(101),중화1동,594.0


In [67]:
house_dongbuk = house_dongbuk.groupby(['시군구', '단지명', '행정동'])['면적(㎡)당금액'].mean().round().reset_index()
house_dongbuk

,시군구,단지명,행정동,면적(㎡)당금액
0,강북구,FIRSTVIEW,수유2동,638.0
1,강북구,경남아너스빌,수유3동,804.0
2,강북구,경남아너스빌(1356-0),송중동,1033.0
3,강북구,궁전,우이동,438.0
4,강북구,극동,인수동,696.0
...,...,...,...,...
758,중랑구,현대,면목7동,892.0
759,중랑구,현대아이파크,묵2동,976.0
760,중랑구,현대휴앤미(102동),중화1동,697.0
761,중랑구,현대휴온,면목2동,648.0


In [68]:
house_dongbuk.to_csv('./data/h동북부실거래가단지.csv')
house_dongbuk

,시군구,단지명,행정동,면적(㎡)당금액
0,강북구,FIRSTVIEW,수유2동,638.0
1,강북구,경남아너스빌,수유3동,804.0
2,강북구,경남아너스빌(1356-0),송중동,1033.0
3,강북구,궁전,우이동,438.0
4,강북구,극동,인수동,696.0
...,...,...,...,...
758,중랑구,현대,면목7동,892.0
759,중랑구,현대아이파크,묵2동,976.0
760,중랑구,현대휴앤미(102동),중화1동,697.0
761,중랑구,현대휴온,면목2동,648.0


In [69]:
house_dongbuk = house_dongbuk.groupby(['시군구', '행정동'])['면적(㎡)당금액'].mean().round().reset_index()
house_dongbuk

,시군구,행정동,면적(㎡)당금액
0,강북구,미아동,768.0
1,강북구,번1동,809.0
2,강북구,번2동,834.0
3,강북구,번3동,866.0
4,강북구,삼각산동,1102.0
...,...,...,...
90,중랑구,상봉2동,1057.0
91,중랑구,신내1동,838.0
92,중랑구,신내2동,1043.0
93,중랑구,중화1동,842.0


In [70]:
house_dongbuk.to_csv('./data/h동북부실거래가.csv')
house_dongbuk

,시군구,행정동,면적(㎡)당금액
0,강북구,미아동,768.0
1,강북구,번1동,809.0
2,강북구,번2동,834.0
3,강북구,번3동,866.0
4,강북구,삼각산동,1102.0
...,...,...,...
90,중랑구,상봉2동,1057.0
91,중랑구,신내1동,838.0
92,중랑구,신내2동,1043.0
93,중랑구,중화1동,842.0
